In [7]:
import sys

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scanpy as sc

sys.path.append("..")
# this is what you need for python script
# sys.path.append(os.path.abspath(os.path.join(os.path.dirname(__file__), '..')))

from partipy.paretoti_funcs import var_explained_aa, set_dimension

plt.style.use("dark_background")

In [8]:
def simulate(
    n_samples: int,
    n_archetypes: int,
    n_dimensions: int,
    noise_std: float,
    seed: int = 42,
):
    assert noise_std >= 0

    rng = np.random.default_rng(seed=seed)
    Z = rng.uniform(low=-1, high=1, size=(n_archetypes, n_dimensions))
    A = rng.exponential(scale=1, size=(n_samples, n_archetypes))
    A /= A.sum(axis=1, keepdims=True)
    X = A @ Z
    if noise_std > 0:
        X += rng.normal(loc=0, scale=noise_std, size=X.shape)

    assert np.all(np.isclose(A.sum(axis=1), 1))
    assert A.min() > 0

    return X, A, Z

In [9]:
X, A, Z = simulate(
    n_samples=5_000,
    n_archetypes=3,
    n_dimensions=7,
    noise_std=0.1
    )
X.shape

(5000, 7)

In [10]:
adata = sc.AnnData(obs=pd.DataFrame(index=np.arange(X.shape[0])), obsm={"X_pca": X})
set_dimension(adata, 7)

/Users/pschafer/miniforge3/envs/partipy/lib/python3.11/site-packages/anndata/_core/aligned_df.py:68: ImplicitModificationWarning: Transforming to str index.


In [11]:
var_explained_aa(adata=adata,
                 min_a=2,
                 max_a=10,
                 n_jobs=1)

In [12]:
var_explained_aa(adata=adata,
                 min_a=2,
                 max_a=10,
                 n_jobs=-1)